# 주식 및 ETF 데이터 수집

In [1]:
# pip install beautifulsoup4 pandas openpyxl selenium webdriver-manager pykrx sqlalchemy pymysql


In [1]:
# pip install pymysql


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'e:\아경\AI\Python38\python.exe -m pip install --upgrade pip' command.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from pykrx import stock
import datetime
import os
from sqlalchemy import create_engine
import pymysql
import numpy as np
import configparser


In [2]:


# 설정 파일 읽기
config = configparser.ConfigParser()
config.read('config.ini')

username = config['mysql']['username']
password = config['mysql']['password']
host = config['mysql']['host']
database = config['mysql']['database']

conn = pymysql.connect(host=host, user=username, password=password, db=database, charset='utf8')
cur = conn.cursor()

# 테이블 생성 함수
def create_table():
    cur.execute('''
    CREATE TABLE IF NOT EXISTS stock_data (
        ticker VARCHAR(10),
        name VARCHAR(100),
        date DATE,
        open FLOAT,
        high FLOAT,
        low FLOAT,
        close FLOAT,
        volume BIGINT,
        value BIGINT,
        market_cap BIGINT,
        shares_outstanding BIGINT,
        PER FLOAT,
        PBR FLOAT,
        dividend FLOAT,
        normalized_value FLOAT,
        PRIMARY KEY (ticker, date)
    )
    ''')

# 테이블 생성
create_table()

In [10]:
# 조건 설정
per_threshold = 10
pbr_threshold = 1
div_threshold = 3.5  # 기준금리 참조

# 현재 날짜 기준으로 최근 월 종목 리스트 수집
current_date = datetime.datetime.now()
last_month_str = current_date.strftime("%Y%m") + '01'

tickers_kosdaq = stock.get_market_ticker_list(last_month_str, market='KOSDAQ')
tickers_kospi = stock.get_market_ticker_list(last_month_str, market='KOSPI')
recent_tickers = set(tickers_kosdaq + tickers_kospi)

# 각 티커에 대한 처리
for ticker in recent_tickers: #all_tickers:처음엔 all_tickers로.
    # 티커의 마지막 자리가 0이 아니면 건너뜀
    if ticker[-1] != '0':
        continue

    # 데이터베이스에서 마지막 날짜 확인
    cur.execute(f"SELECT MAX(date) FROM stock_data WHERE ticker = '{ticker}'")
    last_date = cur.fetchone()[0]

    # 새 데이터를 가져옴
    if last_date:
        start_date = (last_date + datetime.timedelta(days=1)).strftime("%Y%m%d")
    else:
        start_date = "19800102"

    end_date = current_date.strftime("%Y%m%d")

    df2 = stock.get_market_fundamental(start_date, end_date, ticker)
    if df2.empty or 'PER' not in df2.columns:
        print('df2.empty')
        continue

    # 조건 확인
    if ('PER' in df2.columns and 'PBR' in df2.columns and 'DIV' in df2.columns):
        condition = (df2['PER'] > 0) & (df2['PER'] <= per_threshold) & \
                    (df2['PBR'] > 0) & (df2['PBR'] <= pbr_threshold) & \
                    (df2['DIV'] >= div_threshold)
        if not condition.any():
             print('no condition')
            continue  # 조건을 만족하지 않으면 다음 티커로 넘어감
        
    else:
        print('no columns')
        continue  # 필요한 컬럼이 없으면 넘어감

    df1 = stock.get_market_ohlcv(start_date, end_date, ticker)
    if df1.empty:
        print('df1.empty')
        continue

    # '고가' 컬럼이 있는지 확인
    if '고가' not in df1.columns:
        print('no고가')
        continue
    df3 = stock.get_market_cap(start_date, end_date, ticker)
    combined_df = pd.concat([df1, df2, df3], axis=1, join='outer')
    
    
    # 종목명 가져오기
    name = stock.get_market_ticker_name(ticker)


    # 노멀라이즈드 값 계산
    combined_df['normalized_value'] = (combined_df['종가'] - combined_df['종가'].rolling(window=252*5, min_periods=252).min()) / \
                                      (combined_df['종가'].rolling(window=252*5, min_periods=252).max() - combined_df['종가'].rolling(window=252*5, min_periods=252).min()) * 100

    combined_df.reset_index(inplace=True)
    combined_df['ticker'] = ticker
    combined_df['name'] = name
    combined_df.rename(columns={
        '날짜': 'date',
        '시가': 'open',
        '고가': 'high',
        '저가': 'low',
        '종가': 'close',
        '거래량': 'volume',
        '거래대금': 'value',
        '시가총액': 'market_cap',
        '상장주식수': 'shares_outstanding',
        'DIV': 'dividend'
    }, inplace=True)

    # 중복된 'volume' 열 제거
    combined_df = combined_df.loc[:,~combined_df.columns.duplicated()]

    # NaN 값을 None으로 변환 (개별적으로 처리)
    combined_df = combined_df.replace({np.nan: None})

    # 데이터베이스에 저장
    for _, row in combined_df.iterrows():
        sql = '''
        INSERT INTO stock_data (ticker, name, date, open, high, low, close, volume, value, market_cap, shares_outstanding, PER, PBR, dividend, normalized_value)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            open=VALUES(open),
            high=VALUES(high),
            low=VALUES(low),
            close=VALUES(close),
            volume=VALUES(volume),
            value=VALUES(value),
            market_cap=VALUES(market_cap),
            shares_outstanding=VALUES(shares_outstanding),
            PER=VALUES(PER),
            PBR=VALUES(PBR),
            dividend=VALUES(dividend),
            normalized_value=VALUES(normalized_value)
        '''
        cur.execute(sql, (
            row['ticker'],
            row['name'],
            row['date'].strftime('%Y-%m-%d') if row['date'] else None, 
            row['open'], 
            row['high'], 
            row['low'], 
            row['close'], 
            int(row['volume']) if row['volume'] is not None else None, 
            int(row['value']) if row['value'] is not None else None, 
            int(row['market_cap']) if row['market_cap'] is not None else None, 
            int(row['shares_outstanding']) if row['shares_outstanding'] is not None else None, 
            row['PER'], 
            row['PBR'], 
            row['dividend'], 
            row['normalized_value']
        ))

    conn.commit()
    print(f"{ticker} 데이터베이스에 저장 완료")

cur.close()
conn.close()
print("모든 데이터가 성공적으로 저장되었습니다.")

041650 데이터베이스에 저장 완료
067000 데이터베이스에 저장 완료
035510 데이터베이스에 저장 완료
376290 데이터베이스에 저장 완료
052460 데이터베이스에 저장 완료
030960 데이터베이스에 저장 완료
034830 데이터베이스에 저장 완료


KeyboardInterrupt: 

## ETF 티커 & 리스트 생성  

In [6]:
# 크롬 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')  # 브라우저 창을 띄우지 않음
options.add_argument('disable-gpu')  # GPU 가속 비활성화
options.add_argument('lang=ko_KR')  # 한국어 페이지

# 크롬 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [8]:
# URL에서 HTML 컨텐츠를 가져옴
url = 'https://finance.naver.com/sise/etf.naver'
driver.get(url)

# 종목명과 티커를 추출하여 리스트로 저장
etf_data = []
rows = driver.find_elements(By.CSS_SELECTOR, '#etfItemTable > tr')
for row in rows:
    name_col = row.find_elements(By.CSS_SELECTOR, 'td.ctg a')
    if name_col:
        name = name_col[0].text.strip()
        ticker = name_col[0].get_attribute('href').split('=')[-1]  # 종목명 링크에서 티커 추출
        etf_data.append([name, ticker])

# 드라이버 종료
driver.quit()

# DataFrame 생성
df = pd.DataFrame(etf_data, columns=['종목명', '티커'])

# DataFrame을 Excel 파일로 저장
excel_filename = 'etf_list.xlsx'
df.to_excel(excel_filename, index=False)

print(f"Excel 파일로 저장되었습니다: {excel_filename}")


Excel 파일로 저장되었습니다: etf_list.xlsx
